<a href="https://colab.research.google.com/github/Festuskipkoech/SpeakAI/blob/main/SmartFarming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Install Kaggle API
!pip install -q kaggle

# 2. Upload your kaggle.json API key
from google.colab import files
files.upload()  # Upload kaggle.json when prompted

# 3. Setup environment
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 4. Download PlantVillage dataset
!kaggle datasets download -d emmarex/plantdisease

# 5. Unzip
!unzip -q plantdisease.zip -d PlantVillage

# 6. Verify
!ls PlantVillage

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/emmarex/plantdisease
License(s): unknown
 98% 647M/658M [00:05<00:00, 163MB/s]
100% 658M/658M [00:05<00:00, 123MB/s]
plantvillage  PlantVillage


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


In [ ]:
# Configuration
DATASET_PATH = '/content/PlantVillage/PlantVillage'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 15
EPOCHS = 5

def data_preprocessing():
    # Data Augmentation
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2
    )

    # Load and prepare data
    train_generator = train_datagen.flow_from_directory(
        DATASET_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = train_datagen.flow_from_directory(
        DATASET_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, validation_generator


In [ ]:

def create_model():
    # Transfer learning with MobileNetV2
    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=IMAGE_SIZE + (3,)
    )
    base_model.trainable = False  # Freeze base layers

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def train_model():
    # Prepare data
    train_generator, validation_generator = data_preprocessing()

    # Create model
    model = create_model()

    # Callbacks
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=0.00001
    )

    early_stop = EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    )

    # Training
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=validation_generator,
        callbacks=[reduce_lr, early_stop]
    )

    return model


In [ ]:

def model_evaluation(model, validation_generator):
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(validation_generator)
    print(f'Test accuracy: {test_accuracy * 100:.2f}%')

def model_export(model):
    # Convert to TensorFlow Lite for mobile deployment
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    tflite_model = converter.convert()

    # Save the model
    with open('plant_disease_model.tflite', 'wb') as f:
        f.write(tflite_model)


In [ ]:
def main():
    # Full training pipeline
    trained_model = train_model()
    model_export(trained_model)

if __name__ == '__main__':
    main()

Found 16516 images belonging to 15 classes.
Found 4122 images belonging to 15 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


117/517 ━━━━━━━━━━━━━━━━━━━━ 12:40 2s/step - accuracy: 0.2189 - loss: 2.6051

KeyboardInterrupt: 

In [ ]:
!ls